In [1]:
%store -r data_unlabeled
%store -r data_labeled
%store -r preprocessed_df
%store -r imputed_data
%store -r raw_data

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


Using TensorFlow backend.


In [3]:
data_unlabeled = data_unlabeled.drop(["Død", "Ktr_andre", "Oppdaget_annen", "Sympt_annet", "Verif_annet", "Lok_annet", "Hvorfor_ikke"], axis=1)

In [4]:
normal_input = data_unlabeled[["Sympt_blødning", "Sympt_smerter", "Sympt_ascites", "Sympt_fatigue"]]
the_best_input = data_unlabeled[["Age", "Sympt_blødning", "Sympt_smerter", "Sympt_vekttap", "Sympt_utflod", "Sympt_ascites", "Sympt_avføring", "Sympt_ileus", "Sympt_fatigue", "Sympt_vannlating", "Sympt_blodfeces", "Sympt_blodurin", "Sympt_anorexi", "Sympt_kakeksi", "Sympt_hudtumor", "sympt_luftveier"]]

In [5]:
def mk_int(s):
    s = s.strip()
    
    try:
        s = int(s)
    except:
        s = float(s)
    finally:
        return s if s else None

In [6]:
header_names = list(data_unlabeled.columns.values)

In [7]:
data_unlabeled[:10]

,Pas_ID,Født,Oppdaget_når,Age,lever,FIGO_stadium,Substadium,histologi,gradering,kreftform,OVAgrade,myometireinvasjon,Cx_invasjon,Karinvasjon,Lymfekarinvasjon,Lymfeknuter,Tumor_str,hist_nummer,Primærbehandling,Avsluttet_prim_beh,Ant_ktr,Beh_sykehus,Ktr_ingen,Ktr_priv,Ktr_lok,Ktr_Uni,Ktr_komb,Ktr_fastlege,Lengde_sympt_dager,Lengde_sympt_uker,Lengde_sympt_mnd,Symptomer_mindre_1mnd,TidSympt_2dic,Oppdaget_pas,Oppdaget_FL,Oppdaget_Priv,Oppdaget_Lok,Oppdaget_Univ,Oppdaget_Kir,Tidligere_kontakt,Sympt_tilstede,Sympt_blødning,Sympt_smerter,Sympt_vekttap,Sympt_utflod,Sympt_ascites,Sympt_avføring,Sympt_ileus,Sympt_fatigue,Sympt_vannlating,Sympt_blodfeces,Sympt_blodurin,Sympt_anorexi,Sympt_kakeksi,Sympt_hudtumor,sympt_luftveier,Verif_histo,Histo_nummer,Histo_lab,Verif_cytol,Cytol_nummer,Cytol_lab,Verif_MR,Verif_CT,Verif_RTH,Verif_ABD_UL,Verif_VAG_UL,ULverif,Verif_Colon,Lok_vagina,Lok_bekken,Lok_ØvreAbd,Lok_lunger,Lok_lever,Lok_lok_LK,Lok_fjern_LK,Lok_skjelett,Lok_hjernen,Henvist_Uni,Behandling,Time_to_recurrence_months,symptomer_kliniske,KlinSympt_2dic,symptomer,Symtomer_tilstede,Sympt_tilstede2dic,Symptomer_lengde,Distal_recurrence,Local_recurrence,Age_at_diagnosis,age_rec
0,5013,-1034.4708,1356.9948,75,,1,3,1,2,1,,,2,2,,2,,,1,1235.8620,8,5,0,0,0,5,0,0,5,0,0,1,1,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,,1,4156,5,0,1,0,1,1,1,0,0,0,0,0,0,0,1,0,0,2,2,46,1,1,5,2,1,0,1,0,71,75
1,1244,-909.7992,1448.9244,74,1,1,2,1,2,1,,2,2,1,,,,1503931,1,1425.1644,3,,0,0,0,0,,0,0,1,0,1,1,1,0,0,1,1,0,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,15160,1,1,16000088,,0,1,0,1,1,1,0,1,0,1,0,0,0,1,0,0,1,3,9,3,1,1,2,1,0,1,0,73,74
2,2116,-1126.0548,1406.8440,80,,1,3,1,1,1,,,,,,,,819216,5,1230.7644,16,2,0,1,0,0,0,0,0,0,1,0,1,1,0,1,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1427589,2,0,,,0,1,0,1,1,1,0,0,1,0,0,0,0,0,0,0,1,3,67,1,1,1,2,1,2,0,1,74,80
3,5017,-1092.4452,1383.2604,78,,1,1,6,,1,,,,,,,,4884,1,1362.0924,2,5,0,0,0,5,0,0,0,3,0,1,1,0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,632,5,0,,,0,1,0,1,1,1,0,0,1,1,0,0,0,0,0,0,1,3,8,1,1,3,2,1,0,1,0,77,78
4,5039,-1069.1172,1438.3800,79,,3,3,6,,1,,,,,,,,23874,5,1427.8392,1,5,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,17694,5,0,,,0,1,0,1,1,1,0,1,0,0,0,0,0,0,0,0,1,3,4,0,0,0,0,0,1,0,1,79,79
5,11012,-1189.7316,1446.3324,83,1,1,2,1,2,1,,2,2,1,,,,12628,1,1404.1656,6,11,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,,0,,,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,,16,0,0,0,0,0,1,0,1,82,83
6,29014,-1210.2948,1391.2092,82,1,1,2,1,2,1,,2,2,2,3,2,,,5,1320.1020,7,29,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,,1,1643,16,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,,1,27,0,0,0,0,0,1,0,1,80,82
7,1150,-1063.3284,1398.8952,78,,1,2,1,1,1,,2,2,2,3,2,,1307185,1,1375.3080,3,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,12883,8,0,,,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,2,9,0,0,0,0,0,1,0,1,77,78
8,1176,-1126.0548,1414.7964,80,,1,2,1,2,1,,2,2,1,3,3,,,1,1396.3032,2,6,0,0,0,6,0,0,0,0,12,0,1,1,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,13266,6,0,,,0,1,1,0,1,1,0,1,0,0,0,0,0,0,0,0,1,2,7,1,1,12,2,1,13,0,1,79,80
9,1184,-1350.4356,1417.3884,87,,1,2,12,1,1,,2,2,1,,,,1111414,1,1296.5148,10,31,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1500651,,0,,,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,1,2,46,0,0,0,0,0,1,1,0,83,87


In [8]:
a = []
for i in range(len(data_unlabeled.columns)):
    yes = (data_unlabeled.iloc[:,i].values).tolist()
    b = []
    for item in (yes):
        if(type(item)) == int or (type(item)) == float:
            b.append(item)
        else:
            b.append(mk_int(item))
    a.append(b)

In [9]:
df = pd.DataFrame(a)
df = df.transpose()
df.columns = header_names

In [10]:
df.head()

,Pas_ID,Født,Oppdaget_når,Age,lever,FIGO_stadium,Substadium,histologi,gradering,kreftform,OVAgrade,myometireinvasjon,Cx_invasjon,Karinvasjon,Lymfekarinvasjon,Lymfeknuter,Tumor_str,hist_nummer,Primærbehandling,Avsluttet_prim_beh,Ant_ktr,Beh_sykehus,Ktr_ingen,Ktr_priv,Ktr_lok,Ktr_Uni,Ktr_komb,Ktr_fastlege,Lengde_sympt_dager,Lengde_sympt_uker,Lengde_sympt_mnd,Symptomer_mindre_1mnd,TidSympt_2dic,Oppdaget_pas,Oppdaget_FL,Oppdaget_Priv,Oppdaget_Lok,Oppdaget_Univ,Oppdaget_Kir,Tidligere_kontakt,Sympt_tilstede,Sympt_blødning,Sympt_smerter,Sympt_vekttap,Sympt_utflod,Sympt_ascites,Sympt_avføring,Sympt_ileus,Sympt_fatigue,Sympt_vannlating,Sympt_blodfeces,Sympt_blodurin,Sympt_anorexi,Sympt_kakeksi,Sympt_hudtumor,sympt_luftveier,Verif_histo,Histo_nummer,Histo_lab,Verif_cytol,Cytol_nummer,Cytol_lab,Verif_MR,Verif_CT,Verif_RTH,Verif_ABD_UL,Verif_VAG_UL,ULverif,Verif_Colon,Lok_vagina,Lok_bekken,Lok_ØvreAbd,Lok_lunger,Lok_lever,Lok_lok_LK,Lok_fjern_LK,Lok_skjelett,Lok_hjernen,Henvist_Uni,Behandling,Time_to_recurrence_months,symptomer_kliniske,KlinSympt_2dic,symptomer,Symtomer_tilstede,Sympt_tilstede2dic,Symptomer_lengde,Distal_recurrence,Local_recurrence,Age_at_diagnosis,age_rec
0,5013.0,-1034.4708,1356.9948,75.0,NaN,1.0,3.0,1.0,2.0,1.0,NaN,NaN,2.0,2.0,NaN,2.0,NaN,NaN,1.0,1235.8620,8.0,5.0,NaN,0.0,0.0,5.0,NaN,NaN,5.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,4156.0,5.0,0.0,1.0,0.0,1.0,1.0,1.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,2.0,2.0,46.0,1.0,1.0,5.0,2.0,1.0,0.0,1.0,0.0,71.0,75.0
1,1244.0,-909.7992,1448.9244,74.0,1.0,1.0,2.0,1.0,2.0,1.0,NaN,2.0,2.0,1.0,NaN,NaN,NaN,1503931.0,1.0,1425.1644,3.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,15160.0,1.0,1.0,16000088.0,NaN,0.0,1.0,0.0,1.0,1.0,1.0,NaN,1.0,0.0,1.0,NaN,0.0,0.0,1.0,0.0,0.0,1.0,3.0,9.0,3.0,1.0,1.0,2.0,1.0,0.0,1.0,0.0,73.0,74.0
2,2116.0,-1126.0548,1406.8440,80.0,NaN,1.0,3.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,819216.0,5.0,1230.7644,16.0,2.0,NaN,1.0,0.0,0.0,NaN,NaN,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1427589.0,2.0,0.0,NaN,NaN,0.0,1.0,0.0,1.0,1.0,1.0,NaN,0.0,1.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,3.0,67.0,1.0,1.0,1.0,2.0,1.0,2.0,0.0,1.0,74.0,80.0
3,5017.0,-1092.4452,1383.2604,78.0,NaN,1.0,1.0,6.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4884.0,1.0,1362.0924,2.0,5.0,NaN,0.0,0.0,5.0,NaN,NaN,0.0,3.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,632.0,5.0,0.0,NaN,NaN,0.0,1.0,0.0,1.0,1.0,1.0,NaN,0.0,1.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,1.0,3.0,8.0,1.0,1.0,3.0,2.0,1.0,0.0,1.0,0.0,77.0,78.0
4,5039.0,-1069.1172,1438.3800,79.0,NaN,3.0,3.0,6.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23874.0,5.0,1427.8392,1.0,5.0,NaN,0.0,0.0,5.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,17694.0,5.0,0.0,NaN,NaN,0.0,1.0,0.0,1.0,1.0,1.0,NaN,1.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,79.0,79.0


In [24]:
X = df.drop(["kreftform"], axis=1)
y = df[["kreftform"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [26]:
X = X.values
y = y.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [27]:
import tensorflow as tf

encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
onehot_y = np_utils.to_categorical(encoded_y)

In [32]:
y.shape

(680, 1)

In [37]:
from keras.models import Sequential
from keras.layers import Dense,Activation
from keras import optimizers

model = Sequential()
model.add(Dense(5,input_dim=(len(X[0]))))
model.add(Dense(32))
model.add(Dense(3))
model.add(Activation("softmax"))
#Standard learning rate = 0.1
model.compile(loss="mean_squared_error",optimizer="adam",metrics=["accuracy"])


model.fit(X, onehot_y, epochs=100)

Epoch 1/100
680/680 [==============================] - 1s 2ms/step - loss: nan - acc: 0.3132
Epoch 2/100
680/680 [==============================] - 0s 74us/step - loss: nan - acc: 0.3132
Epoch 3/100
680/680 [==============================] - 0s 79us/step - loss: nan - acc: 0.3132
Epoch 4/100
680/680 [==============================] - 0s 89us/step - loss: nan - acc: 0.3132
Epoch 5/100
680/680 [==============================] - 0s 123us/step - loss: nan - acc: 0.3132
Epoch 6/100
680/680 [==============================] - 0s 83us/step - loss: nan - acc: 0.3132
Epoch 7/100
680/680 [==============================] - 0s 78us/step - loss: nan - acc: 0.3132
Epoch 8/100
680/680 [==============================] - 0s 88us/step - loss: nan - acc: 0.3132
Epoch 9/100
680/680 [==============================] - 0s 96us/step - loss: nan - acc: 0.3132
Epoch 10/100
680/680 [==============================] - 0s 95us/step - loss: nan - acc: 0.3132
Epoch 11/100
680/680 [==============================] - 0s 

In [37]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(X_train, y_train.values.ravel())

y_pred = clf.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').